In [1]:
import pandas as pd
import numpy as np

from numpy import savetxt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import sys
np.set_printoptions(threshold=sys.maxsize)
import os
import matplotlib.pyplot as plt
from sklearn import datasets
import glob

from datetime import datetime

def days(d1, d2):
    d1 = datetime.strptime(d1, "%m/%d/%Y")
    d2 = datetime.strptime(d2, "%m/%d/%Y")
    return abs((d2 - d1).days)+1

generate analysis result (MAE, RMSE, MIS, total death)

In [2]:
numpy_vars = {}
i = 0
for np_name in glob.glob('../week55result/*.npz'):
    numpy_vars[i] = np.load(np_name)['prediction']
    i = i +1

In [3]:
ML_ensemble = np.stack([numpy_vars[i] for i in range(20)], axis = 0)
ML_ensemble = np.squeeze(ML_ensemble,axis=2)
print(ML_ensemble.shape)
ML_ensemble = np.mean(ML_ensemble,axis=0)
print(ML_ensemble.shape)

(20, 4, 50, 3)
(4, 50, 3)


In [4]:
# ML_ensemble[0,1]

In [5]:
ML_ensemble_list = []
for i in range (4):
    range_low = ML_ensemble[i,:,1:2] - np.dot(ML_ensemble[i,:,1:2] - ML_ensemble[i,:,0:1],
                                 np.array([[2.576,1.96,1.645,1.282,1.036,0.842,0.674,0.524,0.385,0.253,0.126]]))/1.96

    range_high = ML_ensemble[i,:,1:2] + np.dot(ML_ensemble[i,:,2:3] - ML_ensemble[i,:,1:2],
                                 np.array([[0.126,0.253,0.385,0.524,0.674,0.842,1.036,1.282,1.645,1.96,2.576]]))/1.96
    
    tmp_ML_allquantile = np.concatenate([range_low,ML_ensemble[i,:,1:2],range_high],axis=1)
    ML_ensemble_list.append(tmp_ML_allquantile)
    
ML_ensemble_new = np.stack(ML_ensemble_list,axis=0)
print(ML_ensemble_new.shape)
    

(4, 50, 23)


In [6]:
ML_ensemble_new[0,1]

array([-158.55345727, -121.60200834, -102.70638105,  -80.93142007,
        -66.17483495,  -54.53752798,  -44.45986009,  -35.46194233,
        -27.12387187,  -19.20570424,  -11.58746721,   -4.02921629,
          2.79049392,    9.66432882,   16.80878714,   24.33211825,
         32.45082089,   41.54376784,   52.04395658,   65.3586289 ,
         85.00588928,  102.05516481,  135.3959703 ])

number of crossing

In [7]:
np.sum(np.diff(ML_ensemble_new,axis=2) <= 0)

0

solve crossing problem by setting number larger

In [8]:
# ML_diff = np.diff(ML_ensemble_new,axis=2)
# ML_diff = np.maximum(ML_diff,0)
# ML_diff = np.concatenate([ML_ensemble_new[...,:1],ML_diff],axis=2)
# ML_new_ensemble = np.cumsum(ML_diff,axis=2)

# print(ML_diff.shape)
# print(ML_new_ensemble.shape)

# np.sum(np.diff(ML_new_ensemble,axis=2) ==0)

# np.save('ensembledata.npy', ML_new_ensemble)

In [9]:
np.sum(np.diff(ML_ensemble_new,axis=2) ==0)

0

In [10]:
np.save('ensembledata.npy', ML_ensemble_new)